In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [23]:
#Extract Dangerous Buildings data
dangerous_file = "Resource/Dangerous_Buildings_List.csv"
dangerous_df = pd.read_csv(dangerous_file)
dangerous_df.head(2)

,Case Number,Address,ZIP Code,Case Opened,Kiva PIN,Status of Case,Latitude,Longitude,Location
0,1226669,1104 Ewing Ave,64126,08/20/2019,5246,Pre-Bid Process Ongoing,39.098316,-94.503382,"1104 Ewing Ave\nKansas City, MO 64126\n(39.098..."
1,1180446,5412 E 17th St,64127,06/21/2017,9421,Pre-Bid Process Ongoing,39.090801,-94.518413,"5412 E 17th St\nKansas City, MO 64127\n(39.090..."


In [24]:
#Transform dangerous building DataFrame
limit_dangerous_df = dangerous_df[['Case Number','Case Opened','Status of Case', 'Latitude','Longitude','Location']].copy()
limit_dangerous_df = limit_dangerous_df.rename(columns={"Case Number":"Case_Number","Case Opened":"Opened","Status of Case":"Case_Status"})
# limit_dangerous_df.set_index("Location", inplace = True)
limit_dangerous_df.head(2)

,Case_Number,Opened,Case_Status,Latitude,Longitude,Location
0,1226669,08/20/2019,Pre-Bid Process Ongoing,39.098316,-94.503382,"1104 Ewing Ave\nKansas City, MO 64126\n(39.098..."
1,1180446,06/21/2017,Pre-Bid Process Ongoing,39.090801,-94.518413,"5412 E 17th St\nKansas City, MO 64127\n(39.090..."


In [25]:
#Extract $10M Buildings
ten_mil_file = "Resource/_10M_Demolition_List.csv"
ten_mil_df = pd.read_csv(ten_mil_file)
ten_mil_df.head(2)

,Service Order Number,Date Opened,Structure Status,Address,City,State,Zip Code,Neighborhood,Latitude,Longitude,KivaPIN,Property Owner,Order to Demolish Or Repair Sent,Project Phase,Structure Type,Structure Rating,Mapped Location
0,1149487,12/03/2015,Monitoring Owner Compliance,1115 N Bellefontaine Ave,KANSAS CITY,MO,NaN,NaN,NaN,NaN,38588,Private,NaN,NaN,House,Repair/Receivership,"1115 N Bellefontaine Ave\n(39.128612, -94.544624)"
1,1116561,08/18/2014,Downgraded/No Longer a DB,12504 E 54TH TER,KANSAS CITY,MO,64133.0,Fairway Hills,39.022486,-94.430896,63094,Private,NaN,47/63 Strategic Area,House,Regular Demolition,12504 E 54TH TER\nKANSAS CITY 64133\n(39.02248...


In [26]:
#Transform $10M DataFrame
limit_ten_mil_df = ten_mil_df[['Service Order Number','Date Opened','Structure Status','Neighborhood','Property Owner','Structure Type','Structure Rating','Latitude','Longitude','Mapped Location']].copy()
limit_ten_mil_df = limit_ten_mil_df.rename(columns={"Service Order Number":"Service_Order_Number","Structure Status":"Structure_Status","Property Owner":"Property_Owner","Structure Type":"Structure_Type","Structure Rating":"Structure_Rating",'Date Opened':'Opened','Mapped Location':'Location'})
# limit_ten_mil_df.set_index('Location', inplace = True)
limit_ten_mil_df.head(2)

,Service_Order_Number,Opened,Structure_Status,Neighborhood,Property_Owner,Structure_Type,Structure_Rating,Latitude,Longitude,Location
0,1149487,12/03/2015,Monitoring Owner Compliance,NaN,Private,House,Repair/Receivership,NaN,NaN,"1115 N Bellefontaine Ave\n(39.128612, -94.544624)"
1,1116561,08/18/2014,Downgraded/No Longer a DB,Fairway Hills,Private,House,Regular Demolition,39.022486,-94.430896,12504 E 54TH TER\nKANSAS CITY 64133\n(39.02248...


### Create Database Connection

In [20]:
connection_string = "postgres:postgres@localhost:5432/Building_db"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
# Confirm tables
engine.table_names()

['demo', 'build']

### Load Dataframes into Database

In [27]:
# used replace instead of append. With replace we are creating the tables with given table names in the sql database. 
# Thats why when we confirm the tabeles second time it gives us the table names
limit_dangerous_df.to_sql(name='build', con=engine, if_exists='append', index=False)

In [28]:
limit_ten_mil_df.to_sql(name='demo', con=engine, if_exists='append', index=False)

In [29]:
# Confirm tables
engine.table_names()

['build', 'demo']

### Querying the tables

In [30]:
connection = engine.connect()

In [34]:
build_query = pd.read_sql ("SELECT * FROM build ",connection )

build_query.head(2)

,Case_Number,Opened,Case_Status,Location,Latitude,Longitude
0,1226669,2019-08-20,Pre-Bid Process Ongoing,"1104 Ewing Ave\nKansas City, MO 64126\n(39.098...",39,-95
1,1180446,2017-06-21,Pre-Bid Process Ongoing,"5412 E 17th St\nKansas City, MO 64127\n(39.090...",39,-95


In [36]:
demo_query  = pd.read_sql ("SELECT*FROM demo",connection )

demo_query.head(2)

,Service_Order_Number,Opened,Structure_Status,Neighborhood,Property_Owner,Structure_Type,Structure_Rating,Location,Latitude,Longitude
0,1149487,12/03/2015,Monitoring Owner Compliance,None,Private,House,Repair/Receivership,"1115 N Bellefontaine Ave\n(39.128612, -94.544624)",NaN,NaN
1,1116561,08/18/2014,Downgraded/No Longer a DB,Fairway Hills,Private,House,Regular Demolition,12504 E 54TH TER\nKANSAS CITY 64133\n(39.02248...,39.0,-94.0


In [37]:
# Join tables on location
building_query = pd.read_sql_query ("SELECT Location FROM build", connection)


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "location" does not exist
LINE 1: SELECT Location FROM build
               ^
HINT:  Perhaps you meant to reference the column "build.Location".

[SQL: SELECT Location FROM build]
(Background on this error at: http://sqlalche.me/e/13/f405)

In [16]:
location = pd.read_sql_query ("SELECT Location FROM build", connection)
location 

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "location" does not exist
LINE 1: SELECT Location FROM build
               ^
HINT:  Perhaps you meant to reference the column "build.Location".

[SQL: SELECT Location FROM build]
(Background on this error at: http://sqlalche.me/e/13/f405)